<a href="https://colab.research.google.com/github/Syamchand123/pytorch_handson/blob/main/pytorch_handson_3(RNN_based_QA_system).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import torch

In [5]:
from datasets import load_dataset
data = load_dataset('Abhaykoul/Ancient-Indian-Wisdom')
print(data)

README.md:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/616 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 616
    })
})


In [6]:
data_df=data["train"].to_pandas()
data_df.drop("input", axis=1,inplace=True)
data_df=data_df.rename(columns={"instruction":"question"})
data_df=data_df.rename(columns={"output":"answer"})
order=["question","answer"]
data_df=data_df[order]
data_df['combined_text'] = data_df['question'] + " " + data_df['answer']
data_df

,question,answer,combined_text
0,"In Yoga philosophy, what is the significance o...","Ahimsa, or non-violence, is a fundamental prin...","In Yoga philosophy, what is the significance o..."
1,"In Vedanta, what is the relationship between t...","In Vedanta, the relationship between the indiv...","In Vedanta, what is the relationship between t..."
2,"According to Buddhist teachings, what are the ...",The Four Noble Truths are the foundational tea...,"According to Buddhist teachings, what are the ..."
3,Jainism emphasizes the importance of non-injur...,"In Jainism, the principle of non-injury, or Ah...",Jainism emphasizes the importance of non-injur...
4,"Sikhism promotes the concept of ""langar"" (comm...","In Sikhism, the concept of ""langar"" is a beaut...","Sikhism promotes the concept of ""langar"" (comm..."
...,...,...,...
611,"In what ways does the principle of ""seva"" enco...","The principle of ""seva"" is deeply rooted in th...","In what ways does the principle of ""seva"" enco..."
612,How do the teachings of Vedanta guide us in un...,"In Vedanta, the nature of the self is deeply e...",How do the teachings of Vedanta guide us in un...
613,What role does karma play in shaping our prese...,"Karma, a fundamental concept in Indian spiritu...",What role does karma play in shaping our prese...
614,How can we cultivate inner peace amidst life's...,"In the Yoga philosophy, cultivating inner peac...",How can we cultivate inner peace amidst life's...


In [8]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
def text_to_tokens(text):
 return enc.encode(text)


In [9]:
text_to_tokens(data_df.iloc[0, 1])

[25797,
 5861,
 64,
 11,
 477,
 2536,
 76827,
 768,
 11,
 374,
 264,
 16188,
 17966,
 304,
 38673,
 19675,
 13,
 1102,
 33992,
 603,
 311,
 6725,
 30481,
 323,
 45972,
 7119,
 682,
 5496,
 23837,
 11,
 2737,
 13520,
 13,
 16770,
 5861,
 64,
 2289,
 7953,
 19914,
 2101,
 505,
 7106,
 11682,
 323,
 71010,
 1057,
 11555,
 11,
 4339,
 11,
 323,
 6299,
 13,
 3296,
 99139,
 23128,
 5861,
 64,
 11,
 584,
 37106,
 311,
 1893,
 26348,
 323,
 9096,
 2949,
 323,
 2212,
 603,
 382,
 3947,
 596,
 264,
 6366,
 1370,
 481,
 922,
 23128,
 5861,
 64,
 505,
 279,
 16566,
 370,
 13279,
 460,
 11,
 459,
 14154,
 7904,
 25706,
 13,
 1102,
 596,
 279,
 3446,
 315,
 264,
 39361,
 889,
 11,
 994,
 4691,
 311,
 7124,
 23128,
 5861,
 64,
 11,
 20592,
 11,
 330,
 4599,
 832,
 24886,
 682,
 20566,
 439,
 832,
 596,
 1866,
 659,
 11,
 323,
 279,
 6784,
 323,
 17069,
 315,
 682,
 20566,
 439,
 832,
 596,
 1866,
 11,
 430,
 374,
 279,
 8592,
 1614,
 315,
 23128,
 5861,
 64,
 2266,
 1383,
 56501,
 23128,
 5861,
 64,


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader


class QAdataset(Dataset):

  def __init__(self,df):
    self.df=df;

  def __len__(self):
    return self.df.shape[0]


  def __getitem__(self,index):
    numerical_question = text_to_tokens(self.df.iloc[index]['question'])
    numerical_answer = text_to_tokens(self.df.iloc[index]['answer'])

    # Ensure numerical_question is not empty
    # If text_to_tokens returns an empty list, the RNN will throw an error.
    # We add a placeholder token (e.g., token ID 0) if the question is empty.
    # For production, consider using a specific PAD token or filtering empty questions.
    if not numerical_question:
        numerical_question = [0] # Using 0 as a placeholder token

    # Explicitly cast to torch.long to ensure correct data type for embedding layer
    return torch.tensor(numerical_question, dtype=torch.long), torch.tensor(numerical_answer, dtype=torch.long)

In [11]:
dataset = QAdataset(data_df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [12]:
import torch.nn as nn

class MyRNN(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=512)
    self.rnn=nn.RNN(input_size=512,hidden_size=256,num_layers=1,batch_first=True)
    self.fc=nn.Linear(256,vocab_size)


  def forward(self,question):
    embedded_question=self.embedding(question)
    hidden,final=self.rnn(embedded_question)
    output=self.fc(final.squeeze(0))
    return output

In [13]:
learning_rate=0.001
epochs=100

In [14]:
vocab_size=enc.n_vocab
vocab_size

100277

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model=MyRNN(vocab_size)
model.to(device) # Move model to GPU if available
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Using device: cuda


In [16]:
#training

for epoch in range(epochs):
  total_loss=0
  for question,answer in dataloader:
    # Move tensors to the chosen device (GPU or CPU)
    question = question.to(device)
    answer = answer.to(device)

    optimizer.zero_grad()

    output=model(question)

    loss=criterion(output, answer[:, 0])

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 1314.145732
Epoch: 2, Loss: 778.260615
Epoch: 3, Loss: 722.193671
Epoch: 4, Loss: 660.466593
Epoch: 5, Loss: 610.508986
Epoch: 6, Loss: 557.526979
Epoch: 7, Loss: 499.023185
Epoch: 8, Loss: 450.082127
Epoch: 9, Loss: 395.978663
Epoch: 10, Loss: 349.032510
Epoch: 11, Loss: 292.613715
Epoch: 12, Loss: 253.310142
Epoch: 13, Loss: 218.493790
Epoch: 14, Loss: 158.260029
Epoch: 15, Loss: 162.489799
Epoch: 16, Loss: 160.746807
Epoch: 17, Loss: 130.494919
Epoch: 18, Loss: 134.317488
Epoch: 19, Loss: 115.501151
Epoch: 20, Loss: 100.668851
Epoch: 21, Loss: 90.876866
Epoch: 22, Loss: 103.686915
Epoch: 23, Loss: 119.174990
Epoch: 24, Loss: 103.296919
Epoch: 25, Loss: 100.428189
Epoch: 26, Loss: 104.570480
Epoch: 27, Loss: 110.659247
Epoch: 28, Loss: 71.813891
Epoch: 29, Loss: 60.855940
Epoch: 30, Loss: 56.511170
Epoch: 31, Loss: 55.637162
Epoch: 32, Loss: 52.594107
Epoch: 33, Loss: 50.149570
Epoch: 34, Loss: 78.264804
Epoch: 35, Loss: 117.510136
Epoch: 36, Loss: 54.957494
Epoch: 37

In [3]:
def predict(model , question):

  numeric_question = text_to_tokens(question)
  # Ensure numerical_question is not empty for prediction
  if not numeric_question:
      numeric_question = [0] # Use the same placeholder as in QAdataset

  # Explicitly cast to torch.long to ensure correct data type for embedding layer
  question_tensor = torch.tensor(numeric_question, dtype=torch.long).unsqueeze(0)
  question_tensor = question_tensor.to(device) # Move input tensor to device

  # Set model to evaluation mode
  model.eval()
  with torch.no_grad(): # Disable gradient calculations for prediction
      output = model(question_tensor)
  model.train() # Set model back to training mode if needed for subsequent training

  probs = torch.nn.functional.softmax(output, dim=1)
  value, index = torch.max(probs, dim=1)

  # Use enc.decode to convert the predicted token ID back to text
  # index.item() extracts the scalar value from the tensor
  predicted_token_id = index.item()
  print(enc.decode([predicted_token_id]))

In [19]:
predict(model,"What are the Yamas and Niyamas, and how do they serve as ethical guidelines in yoga?")

The
